# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [1]:
import json

In [2]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [3]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


### Составление корпуса

In [4]:
from gensim import corpora, models
import numpy as np

C:\Users\Asus\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [5]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [6]:
print texts[0]
print corpus[0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [7]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

(3067, u'low-sodium fat-free chicken broth')
(4351, u'sweetened coconut')
(4124, u'baking chocolate')
(197, u'egg roll wrappers')
(6280, u'bottled low sodium salsa')
(1712, u'vegan parmesan cheese')
(5433, u'clam sauce')
(6168, u'mahlab')
(1988, u'(10 oz.) frozen chopped spinach, thawed and squeezed dry')
(1317, u'figs')
(3788, u'caramels')


In [8]:
dictionary.token2id[u'sugar']
?dictionary.token2id[u'sugar']

У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [9]:
np.random.seed(76543)
# здесь код для построения модели:
lda = models.ldamodel.LdaModel(corpus, num_topics=40, passes=5)

In [10]:
show_topics=lda.show_topics(num_topics=40,  num_words=10, formatted=True)

In [11]:
str_text=''
for i in range(40):
    str_text+=show_topics[i][1]+" "

In [12]:
str_text_2=show_topics[0][1]+" "+show_topics[1][1]+" "+show_topics[2][1]+" "+show_topics[3][1]+" "+show_topics[4][1]+" "+show_topics[5][1]+" "+show_topics[6][1]+" "+show_topics[7][1]+" "+show_topics[8][1]+" "+show_topics[9][1]

In [13]:
len(str_text_2)
str_text_2

'0.083*"17" + 0.080*"116" + 0.079*"100" + 0.064*"54" + 0.064*"279" + 0.035*"119" + 0.034*"307" + 0.034*"29" + 0.033*"38" + 0.032*"12" 0.130*"195" + 0.077*"45" + 0.052*"178" + 0.043*"124" + 0.040*"211" + 0.039*"29" + 0.037*"958" + 0.031*"705" + 0.029*"17" + 0.026*"1493" 0.066*"770" + 0.047*"830" + 0.046*"1338" + 0.045*"480" + 0.044*"3" + 0.039*"1637" + 0.035*"816" + 0.033*"806" + 0.030*"117" + 0.030*"1201" 0.117*"41" + 0.112*"207" + 0.081*"17" + 0.050*"383" + 0.049*"0" + 0.048*"45" + 0.046*"155" + 0.030*"29" + 0.029*"425" + 0.029*"397" 0.061*"200" + 0.057*"20" + 0.055*"17" + 0.054*"39" + 0.039*"45" + 0.034*"348" + 0.032*"183" + 0.030*"236" + 0.027*"216" + 0.026*"12" 0.073*"46" + 0.053*"17" + 0.048*"113" + 0.041*"183" + 0.038*"54" + 0.035*"45" + 0.033*"4" + 0.023*"39" + 0.022*"100" + 0.020*"127" 0.113*"51" + 0.087*"366" + 0.056*"310" + 0.052*"494" + 0.048*"387" + 0.046*"541" + 0.034*"373" + 0.033*"249" + 0.031*"52" + 0.030*"511" 0.082*"557" + 0.078*"76" + 0.078*"112" + 0.075*"54" + 0.046

In [14]:
dictionary.token2id["salt"]

17

In [15]:

list=("salt", "sugar", "water", "mushrooms", "chicken", "eggs")
list[0]

'salt'

In [16]:
len(str_text)

5395

In [17]:
str2=str(dictionary.token2id[list[0]])
string ='"'+str2+'"'
string

'"17"'

In [167]:
col_text

[20, 7, 10, 1, 1, 2]

In [18]:
import pandas as pd
col_text=pd.Series([])
col_text=col_text.tolist()
for i in range(len(list)):
    string='"'+str(dictionary.token2id[list[i]])+'"'
    col_text.append(str_text.count(string, 0, len(str_text)))
c_salt=col_text[0]
c_sugar=col_text[1]
c_water=col_text[2]
c_mushrooms=col_text[3]
c_chicken=col_text[4]
c_eggs=col_text[5]
 
c_water


10

In [19]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))
save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs)

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [20]:
import copy
dictionary2 = copy.deepcopy(dictionary)

In [21]:
dict_size_before=len(dictionary2)

In [22]:
list_keys=dictionary2.dfs.keys()
list_values=dictionary2.dfs.values()
#df_1=pd.DataFrame(list(zip(list_keys, list_values)))
df_1=pd.DataFrame({'K':list_keys, 'V':list_values})


In [23]:
df_1_sorted=df_1.sort_values(by=df_1.columns[1])


__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [24]:




df_4000=df_1[df_1.V > 4000]
print(len(df_4000))
df_4000

12


,K,V
0,0,4438
4,4,7380
9,9,4385
12,12,4784
17,17,18048
21,21,4847
29,29,7457
45,45,7972
48,48,6434
54,54,7971


In [25]:
list_to_sort=df_4000.K.tolist()
list_to_sort

[0L, 4L, 9L, 12L, 17L, 21L, 29L, 45L, 48L, 54L, 100L, 117L]

In [26]:
dictionary2.filter_tokens(bad_ids=list_to_sort)

In [27]:
dict_size_after=len(dictionary2)
print(dict_size_before, dict_size_after)

(6714, 6702)


In [28]:
len(corpus)

39774

In [29]:
df5=pd.Series(corpus)


In [30]:
corpus_size_before=len(df5.sum())

In [31]:
corpus_size_before

428249

In [32]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов
df5_after=pd.Series(corpus2)

In [33]:
corpus_size_after=len(df5_after.sum())

In [34]:
corpus_size_after

343665

In [35]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [36]:
save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [153]:
np.random.seed(76543)
# здесь код для построения модели:

lda2 = models.ldamodel.LdaModel(corpus2, id2word=dictionary2, num_topics=40, passes=5)
np.random.seed(76543)
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=40, passes=5)

In [38]:
top_topics2=lda2.top_topics(corpus2)
top_topics1=lda.top_topics(corpus)

In [115]:
top_topics2

[([(0.07124124025081163, u'cumin seed'),
   (0.06739695077061796, u'ground turmeric'),
   (0.05144140749875991, u'garam masala'),
   (0.04891680633804185, u'cinnamon sticks'),
   (0.04554684065165622, u'clove'),
   (0.03772897243038003, u'tomatoes'),
   (0.037189359975672936, u'ginger'),
   (0.033687333094579976, u'black peppercorns'),
   (0.031625122487959005, u'coriander seeds'),
   (0.029635643273047168, u'oil'),
   (0.028615544227769123, u'cilantro leaves'),
   (0.027314969092606552, u'green chilies'),
   (0.02434565597449501, u'tumeric'),
   (0.023891389789773274, u'chili powder'),
   (0.022182448609846545, u'mustard seeds'),
   (0.021275667153493185, u'fennel seeds'),
   (0.020917448639028707, u'ghee'),
   (0.017862781679006855, u'coriander powder'),
   (0.017304522609621915, u'ground cumin'),
   (0.015673903200207125, u'bay leaves')],
  -461.74295128095326),
 ([(0.0799288531089658, u'jalapeno chilies'),
   (0.052438710768818515, u'avocado'),
   (0.05192288980366525, u'lime'),
  

In [39]:
top_topics2_1=[x[1] for x in top_topics2]
coherence2=np.mean(top_topics2_1)
top_topics1_1=[x[1] for x in top_topics1]
coherence=np.mean(top_topics1_1)

In [40]:
print(coherence, coherence2)

(-699.4559739709771, -737.7190246856348)


In [41]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [42]:
save_answers3(coherence, coherence2)

In [43]:
import gensim
print np.__version__
print gensim.__version__

1.16.1
2.3.0


Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [67]:

lda2.get_document_topics(corpus2[0])

[(5, 0.13075619403233654), (7, 0.5054245687730414), (11, 0.2481942371946208)]

Также выведите содержимое переменной *.alpha* второй модели:

In [68]:
lda2.alpha


array([0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
       0.025, 0.025, 0.025, 0.025])

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [154]:
np.random.seed(76543)
# здесь код для построения модели:

lda3 = models.ldamodel.LdaModel(corpus2, id2word=dictionary2, num_topics=40, alpha=1, passes=5)

In [156]:
s2=0
for i in corpus2:
    s2+=len(lda2.get_document_topics(i, minimum_probability=0.01))
print(s2)

198126


In [157]:
s3=0
for i in corpus2:
    s3+=len(lda3.get_document_topics(i, minimum_probability=0.01))
print(s3)

1590960


In [142]:
#lda2.minimum_probability = 0.01
#lda3.minimum_probability = 0.01
number_2_docs=lda2.get_document_topics(corpus2, minimum_probability=0.01)
number_3_docs=lda3.get_document_topics(corpus2, minimum_probability=0.01)

In [152]:
len(number_3_docs[1])

40

In [144]:
df_num_3=pd.Series(number_3_docs)
df_num_2=pd.Series(number_2_docs)

ValueError: Wrong number of dimensions. values.ndim != ndim [3 != 1]

In [158]:
count_model2=s2
count_model3=s3

In [159]:
print(count_model2, count_model3)

(198126, 1590960)


In [160]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [161]:
save_answers4(count_model2, count_model3)

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [163]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [171]:
lda2.get_document_topics(corpus2)
df_num_2=pd.Series(lda2.get_document_topics(corpus2))

In [188]:
df_num_2[10259]

[(19, 0.11602141478677078),
 (22, 0.2892857142857142),
 (28, 0.14642857142857138),
 (34, 0.14642857142857138),
 (39, 0.17683572807037196)]

In [180]:
k=0
len(df_num_2[10259])
for i in range(5):
    k+=df_num_2[10259][i]
print k
    

TypeError: unsupported operand type(s) for +=: 'int' and 'tuple'

In [176]:
corpus2[0]

[(1880, 1), (2837, 1), (3225, 1), (3714, 1), (4902, 1), (5729, 1), (6110, 1)]

In [175]:
df_new=pd.DataFrame.from_dict(recipes)
df_new.ingredients[0]

[u'romaine lettuce',
 u'black olives',
 u'grape tomatoes',
 u'garlic',
 u'pepper',
 u'purple onion',
 u'seasoning',
 u'garbanzo beans',
 u'feta cheese crumbles']

In [ ]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 